## Advent of Code

Day 10: Factory.

In [ ]:
from icecream import ic

def factory(description: str):
    pieces = description.split(" ")

    target_lights_str = pieces[0][1:-1]
    target_lights = [light == "#" for light in target_lights_str]
    current_lights = [False for _ in range(len(target_lights_str))]

    toggles_str = pieces[1:-1]
    buttons = []
    for toggle in toggles_str:
        s_cleaned = toggle[1:-1]
        str_numbers = s_cleaned.split(',')
        buttons.append([int(num) for num in str_numbers])

    return target_lights, current_lights, buttons


def press_button(current_lights: list, button: list):
    for light_pos in button:
        current_lights[light_pos] = not current_lights[light_pos]


def fewest(target_lights, current_lights, buttons) -> int:
    candidates = []
    already_seen = []

    if target_lights == current_lights:
        return 0

    # Seed the candidates.
    for button in buttons:
        cl = current_lights.copy()

        press_button(cl, button)

        if cl not in already_seen:
            candidates.append(cl)
            already_seen.append(cl)

    presses = 1

    while target_lights not in candidates:
        presses += 1

        new_candidates = candidates.copy()
        for candidate in candidates:

            for button in buttons:
                cl = candidate.copy()
                press_button(cl, button)

                if cl not in already_seen:
                    already_seen.append(cl)
                    new_candidates.append(cl)

        candidates = new_candidates.copy()

    return presses


target_lights, current_lights, buttons = factory("[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}")
ic(target_lights, current_lights, buttons)

assert target_lights != current_lights
press_button(current_lights, buttons[4])
press_button(current_lights, buttons[5])
assert target_lights == current_lights

target_lights, current_lights, buttons = factory("[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}")
found = fewest(target_lights, current_lights, buttons)
print(found)

target_lights, current_lights, buttons = factory("[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}")
found = fewest(target_lights, current_lights, buttons)
ic(found)


In [ ]:
file = open("input.txt", "r")
contents = file.read()

part1 = 0
for machine_str in contents.split("\n"):
    target_lights, current_lights, buttons = factory(machine_str)
    presses = fewest(target_lights, current_lights, buttons)
    ic(presses)
    part1 += presses

ic(part1)